In [1]:
import pandas as pd
import numpy as np
import sqlite3
import torch
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import pipeline

In [2]:
catalysts = pd.read_csv('./out/final_cats.csv').drop(labels='Unnamed: 0', axis=1)
sec = pd.read_csv('../sec.csv').drop(labels='Unnamed: 0', axis=1)

In [3]:
sec.tail()

,ticker,year,quarter,filing
17403,IRWD,2016,QTR1,.02 Results of Operations and Financial Condit...
17404,XNCR,2015,QTR1,.02 Departure of Directors or Certain Officers...
17405,NGM,2023,QTR1,.02. Results of Operations and Financial Condi...
17406,ATHX,2013,QTR4,.01. Financial Statements and Exhibits. (d) Ex...
17407,MIST,2023,QTR3,.02. Results of Operations and Financial Condi...


In [4]:
catalysts.head()

,ticker,disease,stage,date,catalyst,label
0,SPPI,Non-Hodgkin’s lymphoma,Approved,2009-09-04,"Approved September 4, 2009.",1
1,JAZZ,Fibromyalgia,CRL,2010-10-11,"CRL received October 11, 2010.",0
2,ASRT,Postherpetic neuralgia - shingles,Approved,2011-01-28,"Approved January 28, 2011.",1
3,ASRT,Colorectal cancer,Approved,2011-04-29,"Approved April 29, 2011.",1
4,SPPI,Colorectal cancer,Approved,2011-04-29,"Approved April 29, 2011.",1


In [5]:
tix_sec = sec.ticker.unique().tolist()
tix_catalysts = catalysts.ticker.unique().tolist()
tix_match = list(set(tix_catalysts).intersection(tix_sec))

In [6]:
len(tix_match)

602

In [7]:
catalysts_filt = catalysts.loc[catalysts.ticker.isin(tix_match)].copy()
sec_filt = sec.loc[sec.ticker.isin(tix_match)].copy()

In [8]:
tokenizer = AutoTokenizer.from_pretrained("gtfintechlab/FOMC-RoBERTa", do_lower_case=True, do_basic_tokenize=True)

model = AutoModelForSequenceClassification.from_pretrained("gtfintechlab/FOMC-RoBERTa", num_labels=3)

config = AutoConfig.from_pretrained("gtfintechlab/FOMC-RoBERTa")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, config=config, device=0, framework="pt")

AssertionError: Torch not compiled with CUDA enabled